In [235]:
# load credentials from environment variables
%load_ext dotenv
%load_ext line_profiler
%load_ext memory_profiler
%dotenv

from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers

import time
import geojson
import geopandas as gpd
import pandas as pd

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [253]:
request_collector = Observers.request_dumper()

geometries = {
    'ooe': (14.318395625014208,48.359153946428485,14.506361110382839,48.543395524629055)
}

with open('../geodata/municipalities_sample.geojson') as f:
    data = geojson.loads(f.read())
    
gdf = gpd.read_file('../geodata/municipalities_sample.geojson')
gdf['bbox'] = gdf.bounds.apply(tuple, axis=1)
gdf['wkt'] = gdf.apply(lambda f: f.geometry.wkt, axis=1)

geometries = [{'name': feat.properties['name'], 'bbox': feat.bbox} for feat in data.features]

In [251]:

def write_cube(feature, time_range):
    '''
    This will write a zarr file using the specified parameters.
    
    # Input
    
    tuple(*attributes) feature
    list[date_string start, date_string end] time_range
    
    
    # Example: 
    
    write_cube(
        Pandas(
            Index=0,
            id='10604',
            name='Loipersbach im Burgenland',
            bbox=(16.446445736463346, 47.680841561177864, 16.49776618971013, 47.72587417451863),
            wkt='MULTIPOLYGON (((16.47614442528707 47.71419149821737,...)))
        ),
        ['2018-01-01', '2018-01-31']
    )
    '''
    
    bands = ['B04']
    config = CubeConfig(
        dataset_name="S2L2A",
        band_names=bands,
        geometry=feature.bbox,
        #tile_size=[1024, 12],
        crs='http://www.opengis.net/def/crs/EPSG/0/4326',
        spatial_res=0.000134,
        time_range=time_range,
        time_period='2W'
    )
    open_cube(config, observer=request_collector).to_zarr(f"{feature.name}_{'_'.join(time_range)}_{time.time()}.zarr")

In [271]:
!rm -r *.zarr
times = []

for feat in gdf.drop('geometry', axis=1).itertuples():
    tr = %timeit -n 1 -r 3 -o write_cube(feat, ['2018-01-01', '2018-01-31'])
    times.append({**vars(tr), 'mean': tr.average, 'stdev': tr.stdev})

!du *.zarr -s | sort -h


Received chunk (0, 0, 0) for band B04: took 239.04 ms
Received chunk (1, 0, 0) for band B04: took 315.64 ms
Received chunk (2, 0, 0) for band B04: took 284.38 ms
Received chunk (0, 0, 0) for band B04: took 315.87 ms
Received chunk (1, 0, 0) for band B04: took 411.34 ms
Received chunk (2, 0, 0) for band B04: took 260.03 ms
Received chunk (0, 0, 0) for band B04: took 241.39 ms
Received chunk (1, 0, 0) for band B04: took 394.98 ms
Received chunk (2, 0, 0) for band B04: took 275.98 ms
1.32 s ± 34.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
Received chunk (0, 0, 0) for band B04: took 470.89 ms
Received chunk (2, 0, 0) for band B04: took 431.41 ms
Received chunk (1, 0, 0) for band B04: took 1149.65 ms
Received chunk (0, 0, 0) for band B04: took 556.91 ms
Received chunk (1, 0, 0) for band B04: took 778.22 ms
Received chunk (2, 0, 0) for band B04: took 539.52 ms
Received chunk (1, 0, 0) for band B04: took 436.29 ms
Received chunk (0, 0, 0) for band B04: took 929.50 ms
Received chun

In [250]:
gdf['bbox'] = gdf.bounds.apply(tuple, axis=1)
gdf['wkt'] = gdf.apply(lambda f: f.geometry.wkt, axis=1)
for feat in gdf.drop('geometry', axis=1).itertuples():
    print(feat.name)

Loipersbach im Burgenland
Weißenkirchen in der Wachau
Sankt Stefan ob Stainz
Bludesch
Alberschwende


In [272]:
pd.DataFrame(times)

,_precision,all_runs,best,compile_time,loops,mean,repeat,stdev,timings,worst
0,3,"[1.3560243339743465, 1.3398812499362975, 1.275...",1.275424,0.000069,1,1.323776,3,0.034820,"[1.3560243339743465, 1.3398812499362975, 1.275...",1.356024
1,3,"[1.917747420957312, 1.9890389190986753, 1.7627...",1.762732,0.000064,1,1.889840,3,0.094473,"[1.917747420957312, 1.9890389190986753, 1.7627...",1.989039
2,3,"[2.3751987281721085, 2.1809036191552877, 2.223...",2.180904,0.000065,1,2.260000,3,0.083327,"[2.3751987281721085, 2.1809036191552877, 2.223...",2.375199
3,3,"[2.8423422230407596, 1.8981353328563273, 1.268...",1.268450,0.000067,1,2.002976,3,0.646801,"[2.8423422230407596, 1.8981353328563273, 1.268...",2.842342
4,3,"[1.7756696259602904, 2.144400909775868, 1.6212...",1.621296,0.000066,1,1.847122,3,0.219452,"[1.7756696259602904, 2.144400909775868, 1.6212...",2.144401


In [273]:
gdf

,id,name,geometry,bbox,wkt
0,10604,Loipersbach im Burgenland,"MULTIPOLYGON (((16.47614 47.71419, 16.47663 47...","(16.446445736463346, 47.680841561177864, 16.49...",MULTIPOLYGON (((16.47614442528707 47.714191498...
1,31351,Weißenkirchen in der Wachau,"MULTIPOLYGON (((15.46017 48.38196, 15.45800 48...","(15.410508300027287, 48.363936587636054, 15.49...",MULTIPOLYGON (((15.46016597182544 48.381964736...
2,60348,Sankt Stefan ob Stainz,"MULTIPOLYGON (((15.24401 46.91554, 15.24397 46...","(15.11885537162377, 46.90972705070729, 15.2963...",MULTIPOLYGON (((15.24400945589979 46.915542026...
3,80104,Bludesch,"MULTIPOLYGON (((9.75464 47.18317, 9.75468 47.1...","(9.708095987716343, 47.17603351606003, 9.75751...",MULTIPOLYGON (((9.7546448278179 47.18317428875...
4,80201,Alberschwende,"MULTIPOLYGON (((9.83665 47.43685, 9.83654 47.4...","(9.79859951325426, 47.43164702893876, 9.892573...",MULTIPOLYGON (((9.836646480218613 47.436853450...
